In [1]:
import numpy as np
from cocktail import lstsq, lstsq_ransac

In [2]:
k = 50
size = 10

# k random points in the square [-size/2, size/2) x [-size/2, size/2)
# Each column is a 2D point
X = size * np.random.random(size=(2, k)) - size / 2

# X plus a last row of ones
X = np.concatenate((X, np.ones((1, X.shape[1]))))

In [3]:
sx = 2
sy = 3
tx = -4
ty = 1

# Unobservable 2D affine transformation
A = np.array([
    [sx, 0, tx],
    [0, sy, ty]
])

print(A)

[[ 2  0 -4]
 [ 0  3  1]]


In [4]:
# Transformed points (plus some noise)
Y = np.matmul(A, X)
# Y += 0.01 * np.random.normal(size=Y.shape)

p = 0.5  # Ratio of inliers
r = int(p * k)  # Number of inliers

# Last k - r points are not from y = A*x model but random
Y[:, r:] = size * np.random.random(size=(Y.shape[0], k - r)) - size / 2

In [5]:
constraints = np.empty((2, 3))
constraints[:] = np.nan
constraints[0, 1] = constraints[1, 0] = 0

# This yields a bad model!
lstsq(X.T, Y.T, constraints).T

array([[ 1.33823781e+00, -3.88578059e-16, -2.56842278e+00],
       [ 0.00000000e+00,  1.45412911e+00,  1.21909974e+00]])

In [6]:
num_iter = 100
sample_size = 2
min_num_inliers = 8
tol = 0.05

constraints = np.empty((2, 3))
constraints[:] = np.nan
constraints[0, 1] = constraints[1, 0] = 0

# This yields a good model
lstsq_ransac(X.T, Y.T, num_iter, sample_size, min_num_inliers, tol, constraints).T

array([[ 2.00000000e+00,  2.22044605e-16, -4.00000000e+00],
       [ 2.55945800e-16,  3.00000000e+00,  1.00000000e+00]])